In [3]:
"""Pytorch dataset object that loads MNIST dataset as bags."""

import numpy as np
import torch
import torch.utils.data as data_utils
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

In [53]:
from torchvision.datasets import DatasetFolder
from torchvision.io import read_image
import os

def loader(path: str) -> torch.Tensor:
    return read_image(path)

class MyNestedFolderDataset(DatasetFolder):
    def __init__(self, root='/Users/ezermoysis/Documents/UCL/Year end project/Malaria/Malaria-Detection-in-Blood-Samples/Red_Cell_Morphology/sma_edof', target_folders=None, transform=None, target_transform=None):
        print('here5')

        super(MyNestedFolderDataset, self).__init__(root, loader, extensions='tiff', transform=transform,
                                                    target_transform=target_transform)
        
        self.target_folders = target_folders if target_folders is not None else []

        self.samples = self._find_samples_in_subfolders()

    def _find_samples_in_subfolders(self):
        samples = []
        classes = sorted(entry.name for entry in os.scandir(self.root) if entry.is_dir())
        print(classes)
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        for class_name in classes:
            class_dir = os.path.join(self.root, class_name)
            for subfolder in os.scandir(class_dir):
                if subfolder.is_dir():
                    for target in os.scandir(subfolder):
                        if target.is_file() and target.name.endswith(self.extensions):
                            item = (target.path, class_to_idx[class_name])
                            samples.append(item)
        return samples

In [54]:
class MyBags(data_utils.Dataset):
    def __init__(self, root='Red_Cell_Morphology', target_folders=None, target_number=1, seed=1, train=True):
        self.root = root
        self.target_folders=target_folders
        self.target_number = target_number
        self.train = train
        print('here2')

        self.r = np.random.RandomState(seed)

        if self.train:
            self.train_bags_list, self.train_labels_list = self._create_bags()
        else:
            self.test_bags_list, self.test_labels_list = self._create_bags()

    def _create_bags(self):
        print('here3')

        data_transform = transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # Change normalization parameters as per your dataset
        print('here4')

        dataset = MyNestedFolderDataset(root=self.root, target_folders=self.target_folders, transform=data_transform)
        loader = data_utils.DataLoader(dataset, batch_size=self.num_in_train if self.train else self.num_in_test, shuffle=False)
        print('here6')

        for (batch_data, batch_labels) in loader:
            all_imgs = batch_data
            all_labels = batch_labels

        bags_list = []
        labels_list = []

        for i in range(self.num_bag):
            bag_length = np.int(self.r.normal(self.mean_bag_length, self.var_bag_length, 1))
            if bag_length < 1:
                bag_length = 1

            if self.train:
                indices = torch.LongTensor(self.r.randint(0, self.num_in_train, bag_length))
            else:
                indices = torch.LongTensor(self.r.randint(0, self.num_in_test, bag_length))

            labels_in_bag = all_labels[indices]
            labels_in_bag = labels_in_bag == self.target_number

            bags_list.append(all_imgs[indices])
            labels_list.append(labels_in_bag)

        return bags_list, labels_list

In [55]:
__name__='__main__'

if __name__ == "__main__":

    print('here')
    train_loader = data_utils.DataLoader(MyBags(root='Red_Cell_Morphology',
                                                target_folders=['sma_edof', 'nonsickle-edofed'],
                                                target_number=1,
                                                seed=1,
                                                train=True),
                                         batch_size=1,
                                         shuffle=True)

    print('here')

    len_bag_list_train = []
    mnist_bags_train = 0
    for batch_idx, (bag, label) in enumerate(train_loader):
        len_bag_list_train.append(int(bag.squeeze(0).size()[0]))
        mnist_bags_train += label[0].numpy()[0]
    print('Number positive train bags: {}/{}\n'
          'Number of instances per bag, mean: {}, max: {}, min {}\n'.format(
        mnist_bags_train, len(train_loader),
        np.mean(len_bag_list_train), np.max(len_bag_list_train), np.min(len_bag_list_train)))

here
here2
here3
here4
here5


FileNotFoundError: Found no valid file for the classes SMA_cells. Supported extensions are: tiff